<a href="https://colab.research.google.com/github/jamelof23/Finance/blob/main/890.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the latest version of Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Install required Python packages
!pip install selenium webdriver-manager beautifulsoup4 pandas openpyxl

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import files

def log_message(message):
    """Log messages to a file and print them."""
    with open("/content/log.txt", "a") as log_file:
        log_file.write(f"{message}\n")
    print(message)

def scrape_stock_data(symbol):
    """Scrape insider transactions for a single stock symbol."""
    try:
        # Set up the Chrome options
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run Chrome in headless mode
        options.add_argument('--no-sandbox')  # Bypass OS security model
        options.add_argument('--disable-dev-shm-usage')  # Overcome resource limitations
        options.add_argument('--disable-gpu')

        # Add the custom user-agent string
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                 "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")

        # Use webdriver_manager to get the path to the ChromeDriver
        service = Service(ChromeDriverManager().install())

        driver = webdriver.Chrome(service=service, options=options)
        log_message(f"Processing symbol: {symbol}")

        # Load the Yahoo Finance URL
        url = f'https://finance.yahoo.com/quote/{symbol}/insider-transactions/'
        driver.get(url)

        # Handle the Cookie Consent Banner
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'accept')]"))
            ).click()
            log_message(f"Accepted cookies for {symbol}")
        except Exception:
            log_message(f"No cookie banner for {symbol}")

        # Scroll to ensure all dynamic content loads
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(6)

        # Wait for the table to load
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, "//table"))
        )
        log_message(f"Table loaded successfully for {symbol}")

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tables = soup.find_all('table')

        if not tables:
            log_message(f"No tables found for {symbol}")
            driver.quit()
            return None

        # Extract the relevant table (adjust index if needed)
        table = tables[2] if len(tables) > 2 else tables[0]
        headers = [th.text.strip() for th in table.find_all('th')]
        rows = []
        for tr in table.find_all('tr')[1:]:
            cells = tr.find_all('td')
            row = [cell.text.strip() for cell in cells]
            rows.append(row)

        # Convert to DataFrame
        df = pd.DataFrame(rows, columns=headers)
        driver.quit()
        log_message(f"Data collected for {symbol}")
        return df

    except Exception as e:
        log_message(f"Error retrieving data for {symbol}: {e}")
        return None

def main():
    # Load symbols from your CSV file
    try:
        symbols_df = pd.read_csv('/content/sample_data/Symbols.csv')
    except FileNotFoundError:
        log_message("Symbols.csv not found. Please upload the file.")
        return

    symbol_list = symbols_df['Symbol'].tolist()

    # Initialize an empty DataFrame to store all data
    combined_df = pd.DataFrame()

    for symbol in symbol_list:
        try:
            df = scrape_stock_data(symbol)
            if df is not None:
                df['Symbol'] = symbol  # Add a column for the stock symbol
                combined_df = pd.concat([combined_df, df], ignore_index=True)
                log_message(f"Data for {symbol} collected and added to combined DataFrame.")
        except Exception as e:
            log_message(f"Error processing symbol {symbol}: {e}")
            continue  # Continue with the next symbol

    # Save the combined data to a single Excel file
    output_file = '/content/Insider_Transactions_All_Stocks.xlsx'
    combined_df.to_excel(output_file, index=False)
    log_message(f"All data has been saved to {output_file}")

    # Download the final Excel file
    files.download(output_file)

# Run the main function
main()


--2024-11-08 21:29:33--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.2.190, 142.251.2.93, 142.251.2.136, ...
Connecting to dl.google.com (dl.google.com)|142.251.2.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111445052 (106M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.8’

google-chrome-stabl 100%[===================>] 106.28M   223MB/s    in 0.5s    

2024-11-08 21:29:34 (223 MB/s) - ‘google-chrome-stable_current_amd64.deb.8’ saved [111445052/111445052]

(Reading database ... 124243 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (130.0.6723.116-1) over (130.0.6723.116-1) ...
Setting up google-chrome-stable (130.0.6723.116-1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading sta

KeyboardInterrupt: 

Working right table only tsla selenium

In [ ]:
# Step 1: Install necessary packages
!apt update
!apt install -y chromium-browser chromium-chromedriver
!pip install undetected-chromedriver selenium beautifulsoup4 pandas openpyxl

import time
import pandas as pd
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Step 2: Set up Chrome options
options = uc.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/google-chrome'

# Add user-agent to avoid detection
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")

# Step 3: Initialize the WebDriver using undetected_chromedriver
driver = uc.Chrome(options=options)

# Step 4: Load the Yahoo Finance URL
url = 'https://finance.yahoo.com/quote/TSLA/insider-transactions/'
driver.get(url)

# Step 5: Handle the Cookie Consent Banner
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'accept')]"))
    ).click()
    print("Accepted cookies")
except Exception as e:
    print("No cookie banner found or already dismissed")

# Step 6: Scroll to the bottom to ensure all dynamic content loads
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

# Step 7: Use WebDriverWait to ensure the table is fully loaded
try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, "//table"))
    )
    print("Table loaded successfully")
except Exception as e:
    print("Timeout: Could not find the table.")
    driver.save_screenshot('/content/screenshot_error.png')
    from google.colab import files
    files.download('/content/screenshot_error.png')
    driver.quit()
    raise e

# Step 8: Parse the Webpage Content using BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract all tables
tables = soup.find_all('table')

# Step 9: Check if any tables were found
if not tables:
    driver.quit()
    raise Exception("No tables found on the page.")

# Extract the first table
table = tables[2]

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]

# Extract table rows
rows = []
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all('td')
    row = [cell.text.strip() for cell in cells]
    rows.append(row)

# Convert the data into a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Step 10: Save the DataFrame to an Excel File
output_file = '/content/TSLA_Insider_Transactions.xlsx'
df.to_excel(output_file, index=False)
print(f"Data has been saved to {output_file}")

# Clean up by closing the browser
driver.quit()

# Step 11: Download the Excel File
from google.colab import files
files.download(output_file)


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

worked code table1

In [ ]:
!pip install undetected-chromedriver selenium beautifulsoup4 pandas openpyxl

import time
import pandas as pd
from bs4 import BeautifulSoup
import undetected_chromedriver as uc

# Step 1: Set up undetected_chromedriver
options = uc.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Step 2: Initialize the WebDriver using undetected_chromedriver
driver = uc.Chrome(options=options)

# Step 3: Load the Yahoo Finance URL
url = 'https://finance.yahoo.com/quote/TSLA/insider-transactions/'
driver.get(url)

# Wait for the page to fully load
time.sleep(5)

# Step 4: Parse the Webpage Content
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Step 5: Locate and Extract the Table
tables = soup.find_all('table')

# Check if any tables were found
if not tables:
    raise Exception("No tables found on the page.")

# Extract the first table
table = tables[0]

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]

# Extract table rows
rows = []
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all('td')
    row = [cell.text.strip() for cell in cells]
    rows.append(row)

# Convert the Data into a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Save the DataFrame to an Excel File
output_file = '/content/TSLA_Insider_Transactions.xlsx'
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")

# Clean up by closing the browser
driver.quit()


Exception: No tables found on the page.

Updated Code with WebDriver Wait

In [ ]:
!pip install undetected-chromedriver selenium beautifulsoup4 pandas openpyxl

import time
import pandas as pd
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Step 1: Set up undetected_chromedriver
options = uc.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Step 2: Initialize the WebDriver using undetected_chromedriver
driver = uc.Chrome(options=options)

# Step 3: Load the Yahoo Finance URL
url = 'https://finance.yahoo.com/quote/TSLA/insider-transactions/'
driver.get(url)

# Step 4: Use WebDriverWait to wait for the table to be visible
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//h3[text()='Insider Transactions Reported - Last Two Years']"))
    )
except Exception as e:
    print("Timeout: Could not find the table title.")
    driver.quit()

# Step 5: Parse the Webpage Content
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Step 6: Locate the table with the title "Insider Transactions Reported - Last Two Years"
table_title = soup.find('h3', string='Insider Transactions Reported - Last Two Years')

if not table_title:
    print("Could not find the table titled 'Insider Transactions Reported - Last Two Years'.")
    driver.quit()

# Step 7: Locate the parent table element following the title
table = table_title.find_next('table')

if not table:
    print("No table found under the specified heading.")
    driver.quit()

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]

# Extract table rows
rows = []
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all('td')
    row = [cell.text.strip() for cell in cells]
    rows.append(row)

# Step 8: Convert the Data into a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Step 9: Save the DataFrame to an Excel File
output_file = '/content/TSLA_Insider_Transactions_Last_Two_Years.xlsx'
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")

# Clean up by closing the browser
driver.quit()

# Step 10: Download the Excel File
from google.colab import files
files.download(output_file)


In [ ]:
!pip install undetected-chromedriver selenium beautifulsoup4 pandas openpyxl

import time
import pandas as pd
from bs4 import BeautifulSoup
import undetected_chromedriver as uc

# Step 1: Set up undetected_chromedriver
options = uc.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Step 2: Initialize the WebDriver using undetected_chromedriver
driver = uc.Chrome(options=options)

# Step 3: Load the Yahoo Finance URL
url = 'https://finance.yahoo.com/quote/TSLA/insider-transactions/'
driver.get(url)

# Wait for the page to fully load
time.sleep(5)

# Step 4: Parse the Webpage Content
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Step 5: Locate the table with the title "Insider Transactions Reported - Last Two Years"
table_title = soup.find('h3', string='Insider Transactions Reported - Last Two Years')

if not table_title:
    raise Exception("Could not find the table titled 'Insider Transactions Reported - Last Two Years'.")

# Step 6: Locate the parent table element following the title
table = table_title.find_next('table')

# Check if the table is found
if not table:
    raise Exception("No table found under the specified heading.")

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]

# Extract table rows
rows = []
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all('td')
    row = [cell.text.strip() for cell in cells]
    rows.append(row)

# Step 7: Convert the Data into a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Step 8: Save the DataFrame to an Excel File
output_file = '/content/TSLA_Insider_Transactions_Last_Two_Years.xlsx'
df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")

# Clean up by closing the browser
driver.quit()

from google.colab import files
files.download('/content/TSLA_Insider_Transactions_Last_Two_Years.xlsx')

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=39895): Read timed out. (read timeout=120)

Yahoo

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from google.colab import files

# Step 1: Upload the CSV file containing stock symbols
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Step 2: Read the CSV file to get the list of stock symbols
df_symbols = pd.read_csv(file_name)
stock_symbols = df_symbols['Symbol'].tolist()

# Initialize a list to store the insider trading data
insider_data = []

# Step 3: Scrape insider trading data for each stock symbol
for symbol in stock_symbols:
    url = f'https://finance.yahoo.com/quote/{symbol}/insider-transactions?p={symbol}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the insider trading table
    table = soup.find('table', {'class': 'W(100%) M(0) BdB Bdc($seperatorColor)'})

    if table:
        # Extract table headers
        headers = [header.text for header in table.find_all('th')]

        # Extract table rows
        rows = table.find_all('tr')[1:]  # Skip the header row

        for row in rows:
            cells = row.find_all('td')
            if len(cells) == len(headers):
                data = [cell.text.strip() for cell in cells]
                data.insert(0, symbol)  # Insert the stock symbol at the beginning
                insider_data.append(data)
    else:
        print(f'No insider trading data found for {symbol}')

    # Pause to avoid overwhelming the server
    time.sleep(2)

# Step 4: Create a DataFrame from the collected data
if insider_data:
    columns = ['Symbol'] + headers
    df_insider = pd.DataFrame(insider_data, columns=columns)

    # Step 5: Save the DataFrame to an Excel file
    output_file = 'insider_trading_data.xlsx'
    df_insider.to_excel(output_file, index=False)

    # Provide a link to download the file
    files.download(output_file)
else:
    print('No insider trading data collected.')


Saving Symbols.csv to Symbols (5).csv
No insider trading data found for TSLA
No insider trading data found for AAPL
No insider trading data found for MRNA
No insider trading data found for HOOD
No insider trading data found for PLTR
No insider trading data collected.




1.   today  6 nov , last data was 15 oct available from 1 nov in yahoo
2.   need progress for what is completed how much remaining
3.   need auto download file
4.   G gift for elon why minus should be plus added stovk  no value eventhough he filed 3 months later
5.  S sell G gift and minus what is Buy (try put stock has bought if B appears)




In [ ]:
# Install required libraries
!pip install finnhub-python openpyxl

import finnhub
import pandas as pd
import time
from google.colab import files
import requests

# Replace 'YOUR_API_KEY' with your actual Finnhub API key
API_KEY = 'csle2g1r01qq49fgr7lgcsle2g1r01qq49fgr7m0'

# Initialize Finnhub client
finnhub_client = finnhub.Client(api_key=API_KEY)

# Load symbols from your Excel file
symbols_df = pd.read_csv('/content/sample_data/Symbols.csv')
symbol_list = symbols_df['Symbol'].tolist()  # Assuming the column name is 'Symbol'

# Function to fetch insider transactions with rate limiting
def fetch_insider_transactions(symbols, rate_limit_per_minute=60):
    insider_data = []
    start_time = time.time()
    requests_made = 0
    for idx, symbol in enumerate(symbols):
        try:
            # Fetch insider transactions with '_from' date
            data = finnhub_client.stock_insider_transactions(symbol, _from='2020-01-01')
            # Check if data is available
            if 'data' in data and data['data']:
                for transaction in data['data']:
                    insider_data.append(transaction)
            else:
                print(f"No insider transactions for symbol {symbol}")
        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                print("Rate limit exceeded. Sleeping for a minute.")
                time.sleep(60)
            else:
                print(f"HTTP error occurred for symbol {symbol}: {http_err}")
        except Exception as e:
            print(f"Error fetching data for symbol {symbol}: {e}")

        requests_made += 1
        # Implement rate limiting
        if requests_made >= rate_limit_per_minute:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            start_time = time.time()
            requests_made = 0
        else:
            time.sleep(1)  # Sleep for 1 second between requests

        # Progress update
        if (idx + 1) % 100 == 0:
            print(f"Processed {idx + 1}/{len(symbols)} symbols")
    return insider_data

# Initialize insider_transactions
insider_transactions = []

try:
    # Fetch insider transactions
    insider_transactions = fetch_insider_transactions(symbol_list)

except KeyboardInterrupt:
    print("\nProcess interrupted by user. Saving data collected so far...")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Saving data collected so far...")

finally:
    if insider_transactions:
        # Convert to DataFrame
        df = pd.DataFrame(insider_transactions)

        # Save to Excel file
        excel_file_name = 'insider_transactions.xlsx'
        df.to_excel(excel_file_name, index=False)

        # Download the Excel file to your local computer
        files.download(excel_file_name)
        print(f"Data saved to {excel_file_name} and downloaded to your local computer.")
    else:
        print("No data collected to save.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Symbols.csv'


**next action dont go sec filing code , find api that can work for ozk too**



sec filing directly

In [ ]:
# Install required libraries
!pip install pandas openpyxl beautifulsoup4 requests lxml tqdm

import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from google.colab import files
from tqdm.notebook import tqdm

# Function to get CIK for a given symbol
def get_cik_map():
    # SEC provides a mapping file for tickers to CIKs
    url = 'https://www.sec.gov/files/company_tickers.json'
    headers = {'User-Agent': 'Jamil jamil.arbas@gmail.com'}
    response = requests.get(url, headers=headers)
    data = response.json()
    cik_map = {}
    for item in data.values():
        symbol = item['ticker']
        cik = str(item['cik_str']).zfill(10)
        cik_map[symbol.upper()] = cik
    return cik_map

def parse_form(cik, accession_number, headers):
    try:
        # Remove leading zeros from CIK and dashes from accession number for the directory
        cik_int = int(cik.lstrip('0'))
        accession_number_no_dashes = accession_number.replace('-', '')
        base_url = f"https://www.sec.gov/Archives/edgar/data/{cik_int}/{accession_number_no_dashes}"
        index_url = f"{base_url}/{accession_number}-index.html"
        print(f"Fetching index page: {index_url}")

        response = requests.get(index_url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve index page at {index_url}")
            return None

        index_content = response.content
        soup = BeautifulSoup(index_content, 'html.parser')

        # Find the table containing the document list
        documents_table = soup.find('table', {'class': 'tableFile', 'summary': 'Document Format Files'})
        if not documents_table:
            print("Could not find the documents table on the index page")
            return None

        # Initialize variable for XML file link
        xml_file_link = None

        # Iterate over table rows to find the XML file
        for row in documents_table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) >= 3:
                description = cells[1].get_text(strip=True).lower()
                document_link_tag = cells[2].find('a')
                if document_link_tag:
                    document_link = document_link_tag['href']
                    document_filename = document_link.split('/')[-1]
                    if document_filename.lower().endswith('.xml'):
                        xml_file_link = 'https://www.sec.gov' + document_link
                        print(f"Found XML file link: {xml_file_link}")
                        break

        if not xml_file_link:
            print("Could not find the XML file link in the index page")
            return None

        # Download and parse the XML file
        print(f"Fetching XML file: {xml_file_link}")
        response = requests.get(xml_file_link, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve XML file at {xml_file_link}")
            return None

        xml_content = response.content
        soup = BeautifulSoup(xml_content, 'xml')

        transactions = []
        reporting_owner_tag = soup.find('reportingOwner')
        if reporting_owner_tag and reporting_owner_tag.find('rptOwnerName'):
            reporting_owner = reporting_owner_tag.find('rptOwnerName').text
            print(f"Reporting owner: {reporting_owner}")
        else:
            reporting_owner = ''
            print("No reporting owner found")

        non_derivative_table = soup.find('nonDerivativeTable')
        if non_derivative_table:
            print("Non-derivative transactions found")
            for row in non_derivative_table.find_all('nonDerivativeTransaction'):
                transaction = {}
                transaction['ownerName'] = reporting_owner
                transaction['securityTitle'] = row.find('securityTitle').get_text(strip=True) if row.find('securityTitle') else ''
                transaction['transactionDate'] = row.find('transactionDate').find('value').text if row.find('transactionDate') else ''
                transaction['transactionCode'] = row.find('transactionCoding').find('transactionCode').text if row.find('transactionCoding') else ''
                transaction['transactionShares'] = row.find('transactionShares').find('value').text if row.find('transactionShares') else ''
                transaction['transactionPricePerShare'] = row.find('transactionPricePerShare').find('value').text if row.find('transactionPricePerShare') else ''
                transaction['sharesOwnedFollowingTransaction'] = row.find('postTransactionAmounts').find('sharesOwnedFollowingTransaction').find('value').text if row.find('postTransactionAmounts') else ''
                transactions.append(transaction)
            print(f"Parsed {len(transactions)} transactions")
        else:
            print("No non-derivative transactions found in the filing")
            return None

        return transactions
    except Exception as e:
        print(f"Error parsing form with accession number {accession_number}: {e}")
        return None


# Function to fetch insider transactions from SEC EDGAR
def fetch_insider_transactions(cik_map):
    insider_data = []
    symbols_with_data = []
    symbols_without_data = []
    headers = {'User-Agent': 'Your Name your.email@example.com'}
    total = len(cik_map)
    requests_made = 0
    rate_limit_per_second = 10  # SEC requests no more than 10 requests per second
    try:
        for symbol, cik in tqdm(cik_map.items(), total=total):
            try:
                # Print the symbol being processed
                print(f"\nProcessing symbol {symbol} ({requests_made + 1}/{total})")
                # Fetch company submissions
                url = f'https://data.sec.gov/submissions/CIK{cik}.json'
                response = requests.get(url, headers=headers)
                data = response.json()

                # Extract filings
                filings = data.get('filings', {}).get('recent', {})
                df = pd.DataFrame(filings)

                if not df.empty:
                    # Filter for Forms 3, 4, 5 (insider transactions)
                    insider_forms = df[df['form'].isin(['3', '4', '5'])]

                    if not insider_forms.empty:
                        data_found = False
                        # Limit to the last 5 filings to reduce processing time
                        insider_forms = insider_forms.head(5)
                        for index, row in insider_forms.iterrows():
                            accession_number = row['accessionNumber']
                            filing_date = row['filingDate']
                            form_type = row['form']

                            # Parse the Form
                            form_data = parse_form(cik, accession_number, headers)
                            if form_data:
                                data_found = True
                                for transaction in form_data:
                                    transaction['symbol'] = symbol
                                    transaction['filingDate'] = filing_date
                                    transaction['formType'] = form_type
                                    insider_data.append(transaction)
                        if data_found:
                            print(f"Insider data found for symbol {symbol}")
                            symbols_with_data.append(symbol)
                        else:
                            print(f"No insider transactions data parsed for symbol {symbol}")
                            symbols_without_data.append(symbol)
                    else:
                        print(f"No insider forms (3, 4, 5) found for symbol {symbol}")
                        symbols_without_data.append(symbol)
                else:
                    print(f"No filings found for symbol {symbol}")
                    symbols_without_data.append(symbol)
            except Exception as e:
                print(f"Error fetching data for symbol {symbol}: {e}")
                symbols_without_data.append(symbol)

            requests_made += 1

            # Implement rate limiting
            if requests_made % rate_limit_per_second == 0:
                time.sleep(1)

        print("\nData fetching completed.")
    except KeyboardInterrupt:
        print("\nProcess interrupted by user. Saving data collected so far...")
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Saving data collected so far...")
    return insider_data, symbols_with_data, symbols_without_data

# Main script execution
if __name__ == '__main__':
    # Get the mapping of symbols to CIKs
    cik_map = get_cik_map()

    # For testing, you can use a subset of symbols (e.g., first 10)
    # subset_cik_map = dict(list(cik_map.items())[:10])
    # For full run, use cik_map
    # insider_transactions, symbols_with_data, symbols_without_data = fetch_insider_transactions(subset_cik_map)

    # Fetch insider transactions for all symbols
    insider_transactions, symbols_with_data, symbols_without_data = fetch_insider_transactions(cik_map)

    # Save to Excel
    if insider_transactions:
        df = pd.DataFrame(insider_transactions)
        excel_file_name = 'insider_transactions.xlsx'
        df.to_excel(excel_file_name, index=False)
        files.download(excel_file_name)
        print(f"Data saved to {excel_file_name} and downloaded to your local computer.")
    else:
        print("No insider transactions data collected.")

    # Optionally, save the lists of symbols with and without data
    with open('symbols_with_data.txt', 'w') as f:
        for symbol in symbols_with_data:
            f.write(f"{symbol}\n")
    with open('symbols_without_data.txt', 'w') as f:
        for symbol in symbols_without_data:
            f.write(f"{symbol}\n")

    # Download the symbol lists
    files.download('symbols_with_data.txt')
    files.download('symbols_without_data.txt')


  0%|          | 0/10077 [00:00<?, ?it/s]


Processing symbol AAPL (1/10077)
Fetching index page: https://www.sec.gov/Archives/edgar/data/320193/000032019324000116/0000320193-24-000116-index.html
Found XML file link: https://www.sec.gov/Archives/edgar/data/320193/000032019324000116/xslF345X05/wk-form4_1729204211.xml
Fetching XML file: https://www.sec.gov/Archives/edgar/data/320193/000032019324000116/xslF345X05/wk-form4_1729204211.xml
No reporting owner found
No non-derivative transactions found in the filing
Fetching index page: https://www.sec.gov/Archives/edgar/data/320193/000032019324000114/0000320193-24-000114-index.html
Found XML file link: https://www.sec.gov/Archives/edgar/data/320193/000032019324000114/xslF345X05/wk-form4_1728426607.xml
Fetching XML file: https://www.sec.gov/Archives/edgar/data/320193/000032019324000114/xslF345X05/wk-form4_1728426607.xml
No reporting owner found
No non-derivative transactions found in the filing
Fetching index page: https://www.sec.gov/Archives/edgar/data/320193/000032019324000112/00003

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

working fantastic missing ozk insider transactions

---



In [ ]:
# Install required libraries
!pip install finnhub-python openpyxl

import finnhub
import pandas as pd
import time
from google.colab import files
import requests

# Replace 'YOUR_API_KEY' with your actual Finnhub API key
API_KEY = 'csle2g1r01qq49fgr7lgcsle2g1r01qq49fgr7m0'

# Initialize Finnhub client
finnhub_client = finnhub.Client(api_key=API_KEY)

data = finnhub_client.stock_insider_transactions('OZK', _from='2020-01-01')
print(data)


data = finnhub_client.stock_insider_transactions('rxrx', _from='2020-01-01')
print(data)


# Function to fetch insider transactions with rate limiting
def fetch_insider_transactions(symbols, rate_limit_per_minute=60):
    insider_data = []
    start_time = time.time()
    requests_made = 0
    for idx, symbol in enumerate(symbols):
        try:
            # Fetch insider transactions with '_from' date
            data = finnhub_client.stock_insider_transactions(symbol, _from='2020-01-01')
            # Check if data is available
            if 'data' in data and data['data']:
                for transaction in data['data']:
                    insider_data.append(transaction)
            else:
                print(f"No insider transactions for symbol {symbol}")
        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 429:
                print("Rate limit exceeded. Sleeping for a minute.")
                time.sleep(60)
            else:
                print(f"HTTP error occurred for symbol {symbol}: {http_err}")
        except Exception as e:
            print(f"Error fetching data for symbol {symbol}: {e}")

        requests_made += 1
        # Implement rate limiting
        if requests_made >= rate_limit_per_minute:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            start_time = time.time()
            requests_made = 0
        else:
            time.sleep(1)  # Sleep for 1 second between requests

        # Progress update
        if (idx + 1) % 100 == 0:
            print(f"Processed {idx + 1}/{len(symbols)} symbols")
    return insider_data

# Initialize insider_transactions
insider_transactions = []

try:
    # Get list of US stock symbols
    symbols_data = finnhub_client.stock_symbols('US')
    symbol_list = [item['symbol'] for item in symbols_data]

    # Fetch insider transactions
    insider_transactions = fetch_insider_transactions(symbol_list)

except KeyboardInterrupt:
    print("\nProcess interrupted by user. Saving data collected so far...")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Saving data collected so far...")

finally:
    if insider_transactions:
        # Convert to DataFrame
        df = pd.DataFrame(insider_transactions)

        # Save to Excel file
        excel_file_name = 'insider_transactions.xlsx'
        df.to_excel(excel_file_name, index=False)

        # Download the Excel file to your local computer
        files.download(excel_file_name)
        print(f"Data saved to {excel_file_name} and downloaded to your local computer.")
    else:
        print("No data collected to save.")


{'data': [], 'symbol': 'OZK'}
{'data': [{'change': -11447, 'currency': '', 'filingDate': '2024-10-17', 'id': '0001601830-24-000179', 'isDerivative': False, 'name': 'Borgeson Blake', 'share': 7089007, 'source': 'sec', 'symbol': 'RXRX', 'transactionCode': 'S', 'transactionDate': '2024-10-15', 'transactionPrice': 6.6678}, {'change': -39375, 'currency': '', 'filingDate': '2024-10-10', 'id': '0001601830-24-000176', 'isDerivative': True, 'name': 'Secora Michael', 'share': 416385, 'source': 'sec', 'symbol': 'RXRX', 'transactionCode': 'M', 'transactionDate': '2024-10-09', 'transactionPrice': 0}, {'change': -15000, 'currency': '', 'filingDate': '2024-10-10', 'id': '0001601830-24-000176', 'isDerivative': False, 'name': 'Secora Michael', 'share': 1450881, 'source': 'sec', 'symbol': 'RXRX', 'transactionCode': 'S', 'transactionDate': '2024-10-09', 'transactionPrice': 6.3583}, {'change': 39375, 'currency': '', 'filingDate': '2024-10-10', 'id': '0001601830-24-000176', 'isDerivative': False, 'name': '